# 4 | _Greykite, Silver_: Autoregress, Daily, Pre-Covid
* [01 API Data Requests](01_API_pulls.ipynb)
* [02 Initial EDA](02_EDA.ipynb)
* [03 Prophet](03_prophet.ipynb)
* [04 Greykite: Silverkite Fuel](04_greykite.ipynb)
* _[04.1 Greykite: Silverkite Fuel](04_greykite_pre.ipynb)_
---

In [11]:
import pandas as pd
from prophet import Prophet

In [12]:
from collections import defaultdict
import pandas as pd
import plotly

from greykite.common.data_loader import DataLoader
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)  # for plots to render in jupyter notebook

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
def date_index(df): 
    df['date'] = pd.to_datetime(df['ds'])
    df = df.set_index('date')
    df.rename(columns = {'ridership' : 'y'}, inplace = True)

    return(df)

In [15]:
# importing bart data
filename = 'bart_daily.csv'
file = '../data/processed/' + filename
bart_df = pd.read_csv(file)

bart_df = date_index(bart_df)

bart_df.head()

,y,ds
date,,
2011-01-01,124162.0,2011-01-01
2011-01-02,93666.0,2011-01-02
2011-01-03,285891.0,2011-01-03
2011-01-04,322306.0,2011-01-04
2011-01-05,327006.0,2011-01-05


In [16]:
# FUNCTION RETURNS PLOTLY TRACES
# TAKES 3 ARGUMENTS: (dataframe, y, and title for plot)
def plot_traces(df, y, title):
    y_trace = go.Scatter(
                    # x = df['date'],
                    x = df.index,
                    y = df[y], 
                    name = y + 'trace',
                    line = dict(color = 'blue'),
                    opacity = 0.4)

    layout = dict(title = title)

    fig = dict(data=[y_trace], layout=layout)
    iplot(fig)
    return (print ('done') )

In [17]:
 # function to output HTML to embed in wordpress
def plot_out(filename, figname):
    import plotly as plt
    out_text = plt.offline.plot(figname, include_plotlyjs=False, output_type='div');

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(out_text)

In [18]:
df = bart_df['2010-01-01':'2020-02-28']
df.columns = ('y', 'ts')
df.head()

,y,ts
date,,
2011-01-01,124162.0,2011-01-01
2011-01-02,93666.0,2011-01-02
2011-01-03,285891.0,2011-01-03
2011-01-04,322306.0,2011-01-04
2011-01-05,327006.0,2011-01-05


In [19]:
# specify dataset information
metadata = MetadataParam(
    time_col = 'ts',    # name of the time column ("date" in example above)
    value_col=  'y',    # name of the value column ("sessions" in example above)
    #freq = 'd'          # "H" for hourly, "D" for daily, "W" for weekly, etc.
                        # Any format accepted by `pandas.date_range` ### USE LOWER CASE OR ERROR for w, m # or remove? dunno why w, m, don't work. 
)

In [20]:
 forecaster = Forecaster()  # Creates forecasts and stores the result
 result = forecaster.run_forecast_config(  # result is also stored as `forecaster.forecast_result`.
     df=df,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=10,  # forecasts 365 steps ahead
         coverage=0.95,         # 95% prediction intervals
         metadata_param=metadata
     )
 )

Fitting 3 folds for each of 1 candidates, totalling 3 fits


In [21]:
ts = result.timeseries
fig = ts.plot()
plotly.io.show(fig)

### GREYKITE EVALUATION 
* creates holdout(test) set by default 
* cross-validation is run on saved data 

In [22]:
 grid_search = result.grid_search
 cv_results = summarize_grid_search_results(
     grid_search=grid_search,
     decimals=2,
     # code below collapse printed output: remove/comment out to show all available metrics and columns.
     cv_report_metrics=None,
     column_order=["rank", "mean_test", "split_test", "mean_train", "split_train", "mean_fit_time", "mean_score_time", "params"])
 # Transposes to save space in the printed output
 cv_results["params"] = cv_results["params"].astype(str)
 cv_results.set_index("params", drop=True, inplace=True)
 cv_results.transpose()

params,[]
rank_test_MAPE,1
mean_test_MAPE,18.59
split_test_MAPE,"(20.2, 13.04, 22.52)"
mean_train_MAPE,14.29
split_train_MAPE,"(6.38, 18.26, 18.22)"
mean_fit_time,12.85
mean_score_time,1.05


 # Backtest: plot the historical forecast on the holdout test set. You can zoom in to see how it performed in any given period.
 backtest = result.backtest
 fig = backtest.plot()
 plotly.io.show(fig)

fname = 'grey_forecast.txt'
plot_out(fname, fig)

In [24]:
 # check historical evaluation metrics (on the historical training/test set).
 backtest_eval = defaultdict(list)
 for metric, value in backtest.train_evaluation.items():
     backtest_eval[metric].append(value)
     backtest_eval[metric].append(backtest.test_evaluation[metric])
 metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
 metrics

,train,test
CORR,0.956138,0.997199
R2,0.914179,0.978937
MSE,1212818566.682476,319301365.380881
RMSE,34825.544744,17869.005719
MAE,22343.699839,14919.646661
MedAE,16290.112717,10004.538489
MAPE,18.252553,5.1393
MedAPE,4.963659,4.489029
sMAPE,4.526941,2.478966
Q80,11289.173691,2983.929332


ID      | MODEL   | DATA      | RMSE        | MSE       | MAE       | CV        | MAPE      | MASE      | AIC 
---     | ---     | ---        | ---       | ---       | ---       | ---       | ---       | ---       | ---  
A       | PROPHET | < 2020   | 446 152   | 199 052 198 567| 375 686   |          
B       | PROPHET | All BART   | 1 243 269   | 5 457 200 928 927| 1 181 450   |          
C       | Greykite| All BART    | 911 443 | 830 729 769 011   |909 234 | | 205
D       | Greykite| < 2020    | 1 053 866| 110 633 739 830  |1 051 141| | 5.25

In [25]:
forecast = result.forecast
fig = forecast.plot()
plotly.io.show(fig)

In [26]:
# The forecasted values are available in `df`

forecast.df.head().round(2)

,ts,actual,forecast,forecast_lower,forecast_upper
0,2011-01-01,124162.0,-19103.83,-83703.05,45495.39
1,2011-01-02,93666.0,42395.58,-26542.77,111333.92
2,2011-01-03,285891.0,289596.00,189249.25,389942.75
3,2011-01-04,322306.0,322233.42,263121.19,381345.65
4,2011-01-05,327006.0,342276.87,282953.57,401600.17


#### Model Diagnostics

The component plot shows how your dataset’s trend, seasonality, and event / holiday patterns are handled in the model:

In [33]:
 fig = forecast.plot_components()
 plotly.io.show(fig)     # fig.show() if you are using "PROPHET" template

fname = 'grey_components.txt'
plot_out(fname, fig)

> Model summary allows inspection of individual model terms. Check parameter estimates and their significance for insights on how the model works and what can be further improved.

In [28]:
 summary = result.model[-1].summary()  # -1 retrieves the estimator from the pipeline
 print(summary)

================================ Model Summary =================================

Number of observations: 3346,   Number of features: 117
Method: Ridge regression
Number of nonzero features: 117
Regularization parameter: 0.4431

Residuals:
         Min           1Q       Median           3Q          Max
  -3.724e+05   -1.526e+04        818.6    1.740e+04    2.157e+05

            Pred_col   Estimate  Std. Err Pr(>)_boot sig. code                    95%CI
           Intercept  3.212e+05    1426.0     <2e-16       ***   (3.183e+05, 3.238e+05)
 events_C...New Year     9813.0 1.158e+04      0.432            (-1.202e+04, 3.212e+04)
 events_C...w Year-1 -1.011e+04 1.500e+04      0.504            (-4.287e+04, 1.562e+04)
 events_C...w Year-2  1.059e+04 1.631e+04      0.528            (-1.627e+04, 4.704e+04)
 events_C...w Year+1    -4350.0 1.438e+04      0.774            (-3.440e+04, 2.461e+04)
 events_C...w Year+2    -2735.0    9660.0      0.768            (-2.391e+04, 1.428e+04)
events_Christ

#### Apply the model

The trained model is available as a fitted `sklearn.pipeline.Pipeline`

In [29]:
 model = result.model
 model

Pipeline(steps=[('input',
                 PandasFeatureUnion(transformer_list=[('date',
                                                       Pipeline(steps=[('select_date',
                                                                        ColumnSelector(column_names=['ts']))])),
                                                      ('response',
                                                       Pipeline(steps=[('select_val',
                                                                        ColumnSelector(column_names=['y'])),
                                                                       ('outlier',
                                                                        ZscoreOutlierTransformer()),
                                                                       ('null',
                                                                        NullTransformer(impute_algorithm='interpolate',
                                                                                        impute_params={'axis': 0,
                                                                                                       'limit_direct...
                                                            'simple_freq': <SimpleTimeFrequencyEnum.DAY: Frequency(default_horizon=30, seconds_per_observation=86400, valid_seas={'WEEKLY_SEASONALITY', 'MONTHLY_SEASONALITY', 'YEARLY_SEASONALITY', 'QUARTERLY_SEASONALITY'})>,
                                                            'start_year': 2011},
                                           uncertainty_dict={'params': {'conditional_cols': ['dow_hr'],
                                                                        'quantile_estimation_method': 'normal_fit',
                                                                        'quantiles': [0.025000000000000022,
                                                                                      0.975],
                                                                        'sample_size_thresh': 5,
                                                                        'small_sample_size_method': 'std_quantiles',
                                                                        'small_sample_size_quantile': 0.98},
                                                             'uncertainty_method': 'simple_conditional_residuals'}))])

In [30]:
 future_df = result.timeseries.make_future_dataframe(
     periods=4,
     include_history=False)
 future_df

,ts,y
2020-02-29,2020-02-29,NaN
2020-03-01,2020-03-01,NaN
2020-03-02,2020-03-02,NaN
2020-03-03,2020-03-03,NaN


> Call .predict() to compute predictions

In [31]:
 model.predict(future_df)

,ts,forecast,forecast_lower,forecast_upper,y_quantile_summary,err_std
0,2020-02-29,174441.195567,109841.979800,239040.411334,"(109841.97980009898, 239040.41133351822)",32959.389191
1,2020-03-01,128951.451338,60013.103498,197889.799177,"(60013.1034980555, 197889.7991774383)",35173.272766
2,2020-03-02,406740.774150,306394.028234,507087.520066,"(306394.0282337195, 507087.5200660543)",51198.260125
3,2020-03-03,446520.254819,387408.022320,505632.487317,"(387408.0223203848, 505632.48731685587)",30159.856489
